In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# **Supplier Performance & Risk Scoring System**

The supplier data containing 10194 records of multiple suppliers with features such as delivery time, quantity, delay, location is used to classify supplier performance from the last 2 year period


In [2]:
#Load the Data
df = pd.read_excel("data.xlsx")

df.head()

,Item,Supplier,PO number,Line value,ETD Original,Invoice Date,Delivery Date Original,Delivery Date registered,SL ID,Del. Index 1,Del. Index 2,Original Quantity,Received on time,Received too late,Not received,Deviation in days,Margin From 1,Margin To 1,Lead time,Transportation time
0,ZZZ,AAA,5223518,46800.00,2024-09-12,2024-09-11,2024-09-12,2024-09-12,VSL 2,100.0,100.0,3200,3200,0,0,0,-10,10,120,0
1,ZZZ,AAA,5223518,46800.00,2024-12-12,2024-12-11,2024-12-12,2024-12-12,VSL 2,100.0,100.0,3200,3200,0,0,0,-10,10,120,0
2,ZZZ,AAA,5223518,46800.00,2025-05-22,2025-05-21,2025-05-22,2025-05-22,VSL 2,100.0,100.0,5800,6000,0,0,0,-10,10,120,0
3,ZZY,AAB,5236406,660.48,2022-09-23,2024-08-29,2024-09-12,2024-09-13,VSL 1,0.0,0.0,3840,0,3840,0,487,-10,10,180,5
4,ZZY,AAB,5243551,660.48,2022-12-05,2024-08-29,2024-09-12,2024-09-13,VSL 1,0.0,0.0,3840,0,3840,0,436,-10,10,180,5


# --- Feature Engineering ---

In [13]:
# Ensure Supplier column is treated as string
df['Supplier'] = df['Supplier'].astype(str).str.strip()

# Normalize SL ID (remove spaces + uppercase)
df['SL ID'] = df['SL ID'].astype(str).str.strip().str.upper()

# Encode Supplier column
le = LabelEncoder()
df['Supplier_encoded'] = le.fit_transform(df['Supplier'])

# Create a decoding map for future reference
decode_map = dict(zip(df['Supplier_encoded'], df['Supplier']))

# Replace NaN with 0 for numeric calculations
for col in ['Received too late', 'Not received', 'Original Quantity', 'Deviation in days per line', 'Lead time', 'Line value']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# --- Map Risk Level based on SL ID ---
df['LR'] = df['SL ID'].map({'VSL 1': 10, 'VSL 2': 20}).fillna(0).astype(int)

# --- STEP 3: Fix negative deviations ---
df.loc[df['Deviation in days'] < 0, 'Deviation in days'] = 0

# --- Compute Weighted Deviation ---
df['Deviation_weighted'] = df['Deviation in days'] * df['Original Quantity']



# --- Supplier Summary ---

In [19]:
# =========================
# STEP 1: Aggregate per Supplier
# =========================
supplier_summary = df.groupby('Supplier').agg(
    Total_Quantity=('Original Quantity', 'sum'),
    Quantity_Received_Too_Late=('Received too late', 'sum'),
    Quantity_Not_Received=('Not received', 'sum'),
    Weighted_Deviation=('Deviation_weighted', 'sum'),  # use precomputed weighted deviation
    Avg_Lead_Time=('Lead time', 'mean'),
    Total_Line_Value=('Line value', 'sum'),
    VSL_Risk=('LR', 'max')  # numeric 10/20/0 from pre-processing
).reset_index()

# Compute Average Delay (AD) per supplier
supplier_summary['AD'] = supplier_summary['Weighted_Deviation'] / supplier_summary['Total_Quantity']

# =========================
# STEP 2: Compute basic ratios
# =========================
supplier_summary['LDR'] = supplier_summary['Quantity_Received_Too_Late'] / supplier_summary['Total_Quantity']
supplier_summary['NRR'] = supplier_summary['Quantity_Not_Received'] / supplier_summary['Total_Quantity']

# Ensure AD is non-negative
supplier_summary['AD'] = supplier_summary['AD'].apply(lambda x: max(x, 0))

# =========================
# STEP 3: Get global min/max for normalization
# =========================
global_min_max = {
    'LDR': supplier_summary['LDR'].min(),
    'LDR_max': supplier_summary['LDR'].max(),
    'NRR': supplier_summary['NRR'].min(),
    'NRR_max': supplier_summary['NRR'].max(),
    'AD': supplier_summary['AD'].min(),
    'AD_max': supplier_summary['AD'].max(),
    'Avg_Lead_Time': supplier_summary['Avg_Lead_Time'].min(),
    'Avg_Lead_Time_max': supplier_summary['Avg_Lead_Time'].max(),
    'Total_Line_Value': supplier_summary['Total_Line_Value'].min(),
    'Total_Line_Value_max': supplier_summary['Total_Line_Value'].max(),
    'VSL_Risk': supplier_summary['VSL_Risk'].min(),
    'VSL_Risk_max': supplier_summary['VSL_Risk'].max()
}

In [20]:
# Compute frequency per supplier
freq_table = df['Supplier'].value_counts().reset_index()
freq_table.columns = ['Supplier', 'Frequency']

# Define quantile bins
quantiles = [0, 0.25, 0.50, 0.75, 0.90, 0.99, 1.0]

freq_table['Supplier_Engagement_Level'] = pd.qcut(
    freq_table['Frequency'],
    q=quantiles,
    labels=[1, 2, 3, 4, 5, 6],  # 1 = lowest, 6 = highest
    duplicates='drop'
)

supplier_summary = supplier_summary.merge(freq_table, on='Supplier', how='left')

supplier_summary['Supplier_Engagement_Level'] = supplier_summary['Supplier_Engagement_Level'].astype(int)



# --- KPI CALCULATIONS ---

In [21]:
# =========================
# STEP 4: Normalize KPIs (0-1), clip to 0-1
# =========================
supplier_summary['LDR_norm'] = ((supplier_summary['LDR'] - global_min_max['LDR']) /
                               (global_min_max['LDR_max'] - global_min_max['LDR'])).clip(0,1)

supplier_summary['NRR_norm'] = ((supplier_summary['NRR'] - global_min_max['NRR']) /
                               (global_min_max['NRR_max'] - global_min_max['NRR'])).clip(0,1)

supplier_summary['AD_norm'] = ((supplier_summary['AD'] - global_min_max['AD']) /
                              (global_min_max['AD_max'] - global_min_max['AD'])).clip(0,1)

supplier_summary['LTR_norm'] = ((supplier_summary['Avg_Lead_Time'] - global_min_max['Avg_Lead_Time']) /
                               (global_min_max['Avg_Lead_Time_max'] - global_min_max['Avg_Lead_Time'])).clip(0,1)

supplier_summary['FE_norm'] = ((supplier_summary['Total_Line_Value'] - global_min_max['Total_Line_Value']) /
                              (global_min_max['Total_Line_Value_max'] - global_min_max['Total_Line_Value'])).clip(0,1)

supplier_summary['VSL_Risk_norm'] = ((supplier_summary['VSL_Risk'] - global_min_max['VSL_Risk']) /
                                     (global_min_max['VSL_Risk_max'] - global_min_max['VSL_Risk'])).clip(0,1)

# =========================
# STEP 5: Round KPIs
# =========================
cols_to_round = ['LDR_norm','NRR_norm','AD_norm','LTR_norm','FE_norm','VSL_Risk_norm']
supplier_summary[cols_to_round] = supplier_summary[cols_to_round].round(4)
supplier_summary['AD'] = supplier_summary['AD'].round(0)

# =========================
# STEP 6: Add global min/max columns
# =========================
supplier_summary['Global_LDR_min'] = global_min_max['LDR']
supplier_summary['Global_LDR_max'] = global_min_max['LDR_max']
supplier_summary['Global_NRR_min'] = global_min_max['NRR']
supplier_summary['Global_NRR_max'] = global_min_max['NRR_max']
supplier_summary['Global_AD_min'] = global_min_max['AD']
supplier_summary['Global_AD_max'] = global_min_max['AD_max']
supplier_summary['Global_LTR_min'] = global_min_max['Avg_Lead_Time']
supplier_summary['Global_LTR_max'] = global_min_max['Avg_Lead_Time_max']
supplier_summary['Global_FE_min'] = global_min_max['Total_Line_Value']
supplier_summary['Global_FE_max'] = global_min_max['Total_Line_Value_max']
supplier_summary['Global_VSL_min'] = global_min_max['VSL_Risk']
supplier_summary['Global_VSL_max'] = global_min_max['VSL_Risk_max']

# =========================
# STEP 7: Reorder columns
# =========================

# Identify all normalized columns
norm_cols = ['LDR_norm','NRR_norm','AD_norm','LTR_norm','FE_norm','VSL_Risk_norm']

# Remove norm columns and engagement from the base list
cols_base = [c for c in supplier_summary.columns 
             if c not in norm_cols and c != 'Supplier_Engagement_Level']

# Final ordering: everything → norm cols → supplier engagement level
supplier_summary = supplier_summary[cols_base + norm_cols + ['Supplier_Engagement_Level']]

# =========================
# STEP 8: Save to Excel
# =========================
output_file = "Supplier_KPI_Normalized_Final.xlsx"
supplier_summary.to_excel(output_file, index=False)
print(f"✅ Supplier KPI saved as: {output_file}")

# =========================
# STEP 9: Preview
# =========================
print(supplier_summary[['Supplier'] + norm_cols + ['Supplier_Engagement_Level']].head())


✅ Supplier KPI saved as: Supplier_KPI_Normalized_Final.xlsx
  Supplier  LDR_norm  NRR_norm  AD_norm  LTR_norm  FE_norm  VSL_Risk_norm  \
0      AAA    0.1810    0.0036   0.0484    0.3183   0.1305            1.0   
1      AAB    0.8105    0.2015   0.8506    0.7318   0.0003            0.0   
2      AAC    0.1850    0.2436   0.0691    0.3072   0.0264            1.0   
3      AAD    0.0229    0.0090   0.0041    0.0168   0.0449            1.0   
4      AAE    0.3805    0.0211   0.1023    0.7329   0.1700            0.0   

   Supplier_Engagement_Level  
0                          5  
1                          2  
2                          5  
3                          5  
4                          5  
